## Setup libraries and working data

In [37]:
import os
import os.path
import time
import yaml
import json
import time
import requests
import shutil
from datetime import datetime

import numpy as pd
import pandas as pd
import geopandas as gpd
import contextily as ctx
import matplotlib.pyplot as plt
from shapely.geometry import Point
import utm
import docker
import openrouteservice
from math import ceil

from dotenv import load_dotenv
load_dotenv()

from tqdm.notebook import tqdm
tqdm.pandas()

%matplotlib inline

DROOT = './data/'
BING_KEY = os.getenv("BING_KEY")

/Users/ivo/opt/anaconda3/envs/thesis-env/lib/python3.7/site-packages/tqdm/std.py:668: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


## Querying & Caching Methods
These modes autoinitialise, except for changing these methods, these should not be opened. 

### Caching method
Drop-in method to save retrieved routes completely locally in the form of a JSON-file.

In [38]:
class Cacher:
    
    def __init__(self, city, dest, mode, dt, debug=False,
                 basepath=os.path.join(DROOT, '3-interim', 'th_routes'),
                 autosave=100):
        """Abstraction method to save route results on disk, so results are
        not lost of the execution is paused. Saves results in individual 
        json files after the number of iterations passes the autosave var.
        
        Parameters
        ----------
        city: string
            The name of the city as in the working csv-file.
        
        dest: shapely.geometry.Point 
            Point of the destination in mercator crs.
            
        mode: string
            The transport mode to be set in filename.
            
        dt: datetime.datetime
            Departure time (naive datetime object). Is set as a key in the json.
        
        basepath: string
            Path to save the objects, preferably an empty directory.
        
        autosave: int / bool
            Autosave after n mutations / manually save using .save().
        
        """
        
        if type(city) is not str:
            raise NameError('Incorrect cityname string given.')
        self.city = city
            
        if mode not in ['Walking', 'Cycling', 'Transit', 'Driving']:
            raise NameError('Incorrect mode given.')
        self.mode = mode
        
        if type(dest) is not Point:
            raise NameError('Incorrect destination point given.')
        self.dest = dest
        self.dest_str = self.point2string(dest)
            
        if type(dt) is not datetime:
            raise NameError('Incorrect datetime object given.')
        self.dt = dt
        self.dt_str = dt.strftime('%Y%m%d-%H%M')
            
        if type(autosave) not in [int, bool] or autosave < 0:
            raise NameError('Incorrect autosave integer given.')
        self.autosave = autosave
        self.mutations = 0

        filename = '{}_{}_{}.json'.format(self.city, self.dest_str, self.mode)
        self.cache_path = os.path.join(basepath, filename)
        
        try:
            with open(self.cache_path) as file:
                self.cache = json.load(file)
        except FileNotFoundError:
            print('{}.json does not exist, created at next cache'.format(city))
            self.cache = {}
            pass
        
        self.debug = debug
            
    def point2string(self, point):
        """Returns a lat,lon formatted string from a shapely.geometry.Point."""
        return "{},{}".format(round(point.y, 5), round(point.x, 5, ))
                  
    def get(self, origin):
        """Returns a single cached result based on a Point."""
        try:
            return self.cache[self.dt_str][self.point2string(origin)]
        except KeyError:
            if self.debug:
                print(origin, f' using key {self.point2string(origin)} not found')
            return False
        
    def get_many(self, origins):
        """Returns all possible cached points in a list of origins."""
        return [self.get(point) for point in origins]
        
    
    def set(self, origin, data):
        """Sets the origin point to a certain dict value."""
        
        if self.dt_str not in self.cache:
            self.cache[self.dt_str] = {}
        
        self.cache[self.dt_str][self.point2string(origin)] = data
        
        self.mutations += 1
        if self.autosave and self.mutations > self.autosave:
            self.save()
            self.mutations = 0
             
        return True
    
    def save(self):
        """
        Manually saves the dict to disk. 
        --> False if there were no mutations to 
            cache object and no saving was required.
        --> True if object was saved successfully.
        """
        
        if self.mutations == 0: 
            return False
        
        with open(self.cache_path, 'w') as file:
            json.dump(self.cache, file)
        return True

Attributes for shapefiles are inspired on the Helsinki Accessibility Research Group, see here: https://blogs.helsinki.fi/accessibility/helsinki-region-travel-time-matrix-2018/




### Driving Query
Current implementation is using Bing Maps Distance Matrix, find [API reference here](https://docs.microsoft.com/en-us/bingmaps/rest-services/routes/calculate-a-distance-matrix).

In [39]:
def drive_query(dest_point, origin_points, dep_time, cityname):
    """Bing Distance Matrix tool. Doesn't work with 
       walking or transit which have to be retrieved one-by-one."""
    
    # Set up caching object.
    cache = Cacher(cityname, dest_point, 'Driving', dep_time)

    # Collect all indexes that are not available in cache yet.
    to_fetch = []
    for i in range(len(origin_points)):
        if cache.get(origin_points[i]) == False:
            to_fetch.append(i)
    
    # Fetch all points that are not in cache yet in batches of 10.
    round_size = 10
    for i in tqdm(range(0, len(to_fetch), round_size),
                  desc='Driving'+str(dep_time.hour), 
                  leave=False):

        # Selection of point indexes (from origin_points) for this round.
        current_points = to_fetch[i:i+round_size]
        
        # Request iternaries from Bing.
        BING_MX_URL = 'https://dev.virtualearth.net/REST/v1/Routes/DistanceMatrix'
        params = {'key': os.getenv("BING_KEY")}
        body = {
            "destinations": [{
                "longitude": dest_point.x,
                "latitude": dest_point.y
            }],
            "origins": [{"longitude": point.x, "latitude": point.y}
                        for point in origin_points[current_points]],
            "travelMode": 'driving',
            "timeUnit": "second",
            "startTime": dep_time.strftime('%m/%d/%Y %-H:%M:%S%p')
        }
        response = requests.post(BING_MX_URL, params=params, json=body)
        
        # Check if response is valid.
        if response.status_code != 200:
            with open(os.path.join(DROOT, 'log.json'), 'w') as file:
                file.write(json.dumps(response.json()))
            raise ValueError('BingError ' + response.json()
                             ['errorDetails'][0])
            
        # Get results from JSON.
        result = response.json()['resourceSets'][0]['resources'][0]
        for iternary in result['results']:
            original_index = current_points[iternary['originIndex']]
            cache.set(origin_points[original_index], iternary)

    # Set up the final dictionary with all points which are now in cache.
    result_dict = {}
    for i in range(len(origin_points)):
        iternary = cache.get(origin_points[i])
        result_dict[i] = {
            "distance": iternary.get('travelDistance', None),
            "travelTime": iternary.get('travelDuration', None)
        }

    cache.save()
    return pd.DataFrame.from_dict(result_dict, orient='index')

# drive_query(origin_point, points, starttime=bingdate)

### Transit & Walking
Current implementation is using Bing Maps Single Route, find [API reference here](https://docs.microsoft.com/en-us/bingmaps/rest-services/routes/calculate-a-route)

In [40]:
# Set up method to retrieve data from Bing in a structured manner.
def transit_walk_query(dest_point, origin_points, mode, dep_time, cityname):
    """Returns route information from Bing API or from cache."""

    # Check if we accept this transport mode.
    assert mode in ['Walking', 'Transit']
    cache = Cacher(cityname, dest_point, mode, dep_time)
    
    # Helper function for converting point to string.
    def pt2latlng(point): return "{lat},{lng}".format(lat=round(point.y, 7), 
                                                      lng=round(point.x, 7))

    # Constants
    BING_EP = 'https://dev.virtualearth.net/REST/v1/Routes/{}'.format(mode)

#     print('code', os.getenv("BING_KEY"))
    
    # Loop over points.
    result_dict = {}
    for point_i in tqdm(range(len(origin_points)), 
                        desc=mode+str(dep_time.hour), 
                        leave=False):

        # Define a this rounds point for use.
        origin_point = origin_points[point_i]
        result = cache.get(origin_point)
        
        # Result is false and not available, we fetch it.
        if not result: # or len(result['resourceSets']) == 0:
            
            # Fetch single item from Bing.
            data = {
                "key": os.getenv("BING_KEY"),
                "waypoint.1": pt2latlng(origin_point),
                "waypoint.2": pt2latlng(dest_point),
                "ra": "routeSummariesOnly",
                "dateTime": dep_time.strftime('%m/%d/%Y %-H:%M:%S%p'),
                "timeType": "Departure",
                "travelMode": mode,
            }
            response = requests.get(BING_EP, params=data)

            # Check if response is valid, raise an error otherwise.
            if response.status_code not in [200, 404, 500]:
                with open(os.path.join(DROOT, 'log.json'), 'w') as file:
                    file.write(json.dumps({
                        'request': data, 
                        "response": response.json(),
                    }))
                raise ValueError('BingError: ' + response.json()
                                 ['errorDetails'][0])
            
            # Write cache to file and set result to the retrieved data.
            result = response.json()
            cache.set(origin_point, result)
            
        # If we have useful results, rename to useful variables. 
        if len(result['resourceSets']) > 0:
            route = result['resourceSets'][0]['resources'][0]
            result_dict[point_i] = {
                "distance": route.get('travelDistance', None),
                "travelTime": route.get('travelDuration', None),
            }

            # If Transit, try to get interval frequency if available.
            if mode == 'Transit':
                try:
                    result_dict[point_i]["intervalTime"] = route['transitFrequencies'][0]['intervalBetweenDepartures']
                except:
                    continue;

    cache.save()
    # Return a dataframe, having converted the dictionary to DataFrame.
    return pd.DataFrame.from_dict(result_dict, orient='index')

### Bike Queries
This implementation relies heavily on locally run a locally run OpenRouteService in a docker. 
* It requires the [OpenRouteService](https://github.com/GIScience/openrouteservice) to be cloned locally and docker to be installed.
* It requires openstreetmap .pbf files (generated in 1.3) and elevation maps (downloaded automatically). 
* It will need to regenerate the docker for every city, which takes about 5 minutes on a 2017 Macbook Pro. 

In [41]:
client = docker.from_env()

# Change this to reflect the local docker instance.
docker_path = '../../openrouteservice/docker/'

docker_config = os.path.join(docker_path, 'docker-compose.yml')
docker_logs = os.path.join(docker_path, 'logs/')
docker_graphs = os.path.join(docker_path, 'graphs/')

# Edit docker-compose file to reflect current city.
def regenerate_ors_docker(cityname):
    
    print("Regenerating ORS-docker, this takes about 5 minutes.")

    with open(docker_config, mode='r') as file:
        conf = yaml.load(file, Loader=yaml.Loader)

    pbf_path_1 = './docker/data/{}.osm.gz'.format(cityname)
    pbf_path_2 = './data/{}.osm.gz'.format(cityname)

    if conf['services']['ors-app']['build']['args']['OSM_FILE'] != pbf_path_1:
    
        conf['services']['ors-app']['build']['args']['OSM_FILE'] = pbf_path_1
        conf['services']['ors-app']['volumes'][-1] = pbf_path_2+':/ors-core/data/osm_file.pbf'

        with open(docker_config, 'w') as file:
            yaml.dump(conf, file, Dumper=yaml.Dumper)

        # Get current container to stop before making changes
        try:
            client.containers.get('ors-app').stop()
        except:
            pass;

        # Remove graphs and logs from other city.
        shutil.rmtree(docker_logs, ignore_errors=True)
        shutil.rmtree(docker_graphs, ignore_errors=True)

    # Set up the new city in a detached mode in Docker.
    !docker-compose -f $docker_config up -d
    
    # Wait for the new city to come online, maximum 10 minutes.
    for i in range(0, 900, 5):
        
        # It might be that the docker is not ready immediately.
        try:
            print(".", end="")
            status = requests.get("http://localhost:8080/ors/health").json()['status']
            if status == 'ready':
                break
        except requests.exceptions.ConnectionError:
            print("-", end ="")
            pass
        
        # Sometimes the Elevation Data still had to be downloaded,
        # and the Container doesn't automatically restart after. We do 
        # this here manually after 7.5 minutes. 
        if i == 450:
            print('restarting..')
            client.containers.get('ors-app').stop()
            !docker-compose -f $docker_config up -d
        
        # Sleep between requests.
        time.sleep(5)
            

In [42]:
def bike_query(dest_point, origin_points, dep_time, cityname):
    
    ors_generated = False
    
    client = openrouteservice.Client(base_url='http://localhost:8080/ors')
    cache = Cacher(cityname, dest_point, 'Cycling', dep_time)
    attempts = 0
    result_dict = {}
    
    for point_i in tqdm(range(len(origin_points)), desc='Biking', leave=False):
        
        origin_point = origin_points[point_i]
        result = cache.get(origin_point)
        
        # Result is false and not available, we fetch it.
        if not result:
            
            if not ors_generated:
                regenerate_ors_docker(cityname)
                ors_generated = True
            
            coords = ((origin_point.x, origin_point.y), (dest_point.x, dest_point.y))
            try:
                result = client.directions(
                    coords, profile='cycling-regular', instructions=False)
            except openrouteservice.exceptions.ApiError:
                result = {'empty': True}
                print('Empty, dumping to cache.')
                    
            cache.set(origin_point, result)
            
        try:
            result_dict[point_i] = {
                'distance': result['routes'][0]['summary']['distance'] / 1000,
                'travelTime': result['routes'][0]['summary']['duration'],
            }
        except KeyError:
            attempts+=1
            pass
        
    cache.save()
    if attempts >= 100:
        print("{}: {} bike routes unavailable".format(cityname, attempts))
    
    return pd.DataFrame.from_dict(result_dict, orient='index')

### Query Point Method
This method loops over all origin points and retrieves routes for all of them using the methods provided. It also recodes them into the proper column names. 

In [43]:
def query_points(dest_point, origin_points, cityname, exclude_modes=[]):

    generated = []
    
    # Departure at Rush-hour (local time 8:00)
    dep_time = datetime(2020, 6, 23, 8)
    
    if 'walking' not in exclude_modes:
        walk_df = transit_walk_query(
            dest_point=dest_point, origin_points=origin_points, 
            mode='Walking', dep_time=dep_time, cityname=cityname)
        walk_df = walk_df.rename(
            columns={'distance': 'walk_d', 'travelTime': 'walk_t'})
        generated.append(walk_df)

    if 'biking' not in exclude_modes:
        bike_df = bike_query(
            dest_point=dest_point, origin_points=origin_points,
            dep_time=dep_time, cityname=cityname)
        bike_df = bike_df.rename(
            columns={'distance': 'bike_d', 'travelTime': 'bike_t'})
        generated.append(bike_df)

    if 'transit' not in exclude_modes:
        trsit_r_df = transit_walk_query(
            dest_point=dest_point, origin_points=origin_points, mode='Transit', 
            dep_time=dep_time, cityname=cityname)
        trsit_r_df = trsit_r_df.rename(
            columns={'distance': 'pt_r_d', 'travelTime': 'pt_r_t', 
                     'intervalTime': 'pt_r_it'})
        generated.append(trsit_r_df)

    if 'driving' not in exclude_modes:
        drive_r_df = drive_query(
            dest_point=dest_point, origin_points=origin_points, 
            dep_time=dep_time, cityname=cityname)
        drive_r_df = drive_r_df.rename(
            columns={'distance': 'car_r_d', 'travelTime': 'car_r_t'})
        generated.append(drive_r_df)

    # Departure at Midday (local time 12:00)
    dep_time = datetime(2020, 6, 23, 12)

    if 'transit' not in exclude_modes:
        trsit_m_df = transit_walk_query(
            dest_point=dest_point, origin_points=origin_points, mode='Transit', 
            dep_time=dep_time, cityname=cityname)
        trsit_m_df = trsit_m_df.rename(
            columns={'distance': 'pt_m_d', 'travelTime': 'pt_m_t', 
                     'intervalTime': 'pt_m_it'})
        generated.append(trsit_m_df)

    if 'driving' not in exclude_modes:
        drive_m_df = drive_query(
            dest_point=dest_point, origin_points=origin_points, 
            dep_time=dep_time, cityname=cityname)
        drive_m_df = drive_m_df.rename(
            columns={'distance': 'car_m_d', 'travelTime': 'car_m_t'})
        generated.append(drive_m_df)

    return generated[0].join(generated[1:])

# Procedures
Loops over all cities specified in citylist.csv and retrieves values for all of them. 

In [44]:
# Get all cities to be retrieved from raw/citylist.csv.
city_df = pd.read_csv(os.path.join(DROOT, '1-raw', 'citylist.csv'))
city_list_df = gpd.GeoDataFrame(
    city_df, crs=4326, 
    geometry=[Point(xy) for xy in zip(city_df.ctr_lon, city_df.ctr_lat)])

In [47]:
index = 0
city_list_itr = city_list_df.itertuples()
for city in city_list_itr:
    
    # Load final outpath.
    outpath_gpkg = os.path.join(DROOT, '3-interim/th_dataframes', city.city + ".gpkg")
    outpath_shp = os.path.join(DROOT, '3-interim/th_dataframes', city.city)
    index += 1

    # If city already exists, skip.
    if os.path.exists(outpath_gpkg) and os.path.exists(outpath_shp) and False:
        print("{}.\t Already created {}".format(index, city.city))
        continue
        
    # Start working on current city:
    gdf_tt = pd.read_pickle(
        os.path.join(DROOT, '3-interim/populationmasks', city.city + '.pcl'))
    
    print('{}.\t Current city:   {}'.format(index, city.city))

    # Get city UTM (meter-distance-accurate EPSG-code).
    zone = utm.from_latlon(city.geometry.y, city.geometry.x)[2]
    utm_epsg = f"326{zone:02d}" if city.geometry.y >= 0 else f"327{zone:02d}"
    
    # Set both GeoDataFrames to correct CRS.
    townhall = city_list_df[city_list_df.city == city.city]
    th_geometry = townhall.to_crs(epsg=utm_epsg).iloc[0].geometry
    gdf_tt = gdf_tt.to_crs(epsg=utm_epsg)
    
    # Declare townhall_geometry as destination.
    gdf_tt['sky_d'] = gdf_tt.distance(th_geometry) / 1000
    gdf_tt = gdf_tt[gdf_tt.sky_d < 10].reset_index()
    
    # If only specific mode is wished, check here.
    excluded = [] #['walking', 'transit', 'driving']
    
    for x in range(5):
        try:
            # Retrieve points using query_points method.
            gdf_tt = gdf_tt.join(
                query_points(city.geometry, gdf_tt.centroid.to_crs(epsg=4326), 
                             city.city, exclude_modes=excluded))
            break
        except requests.exceptions.SSLError:
            # If internet connection error, sleep 30 seconds and restart.
            print("Internet Error, sleeping 30s before restarting.")
            time.sleep(30)
            pass

    # Write out the results to both GeoPackage and Shapefile.
    gdf_tt = gdf_tt.to_crs(epsg=4326)
    gdf_tt.to_file(outpath_shp)
    gdf_tt.to_file(outpath_gpkg, driver="GPKG")


1.	 Current city:   Vienna


2.	 Current city:   Graz


3.	 Current city:   Brussels


4.	 Current city:   Berlin


5.	 Current city:   Hamburg


...

55.	 Current city:   São Paulo


56.	 Current city:   Rio de Janeiro


57.	 Current city:   Salvador


58.	 Current city:   Santiago


59.	 Current city:   Bogota


## Write out a collection of all created dataframes

In [48]:
# Collect all seperate generated dataframes.
dataframes = {
    city: gpd.read_file(
        os.path.join(DROOT, '3-interim/th_dataframes', city + '.gpkg'))
    for city in tqdm(city_list_df.city, leave=True)
    if os.path.isfile(
        os.path.join(DROOT, '3-interim/th_dataframes', city + '.gpkg'))
}

# Concatenate dataframes based on cityname and the dataframe.
gdf_total = pd.concat(dataframes.values(), keys=dataframes.keys())
gdf_total = gdf_total.reset_index().rename(
    columns={'level_0': 'city', 'level_1': 'cell_id'})
gdf_total = gdf_total.drop(columns=['index'])

# Create bins for to group by euclidean distance later.
max_eucl_distance = ceil(gdf_total.sky_d.max())
bins = np.arange(0, max_eucl_distance + 1, 0.5)
gdf_total.insert(
    gdf_total.columns.get_loc('sky_d'),
    'sky_d_bin', 
    pd.cut(x=gdf_total.sky_d, bins=bins, labels=bins[:-1]).astype(float))
    
# Write out to ShapeFile and GeoPackage.
gdf_total.to_file(
    os.path.join('data', '4-processed', 'complete-dataset'), 
    driver="ESRI Shapefile")
gdf_total.to_file(
    os.path.join('data', '4-processed', 'complete-dataset.gpkg'), 
    driver="GPKG")

gdf_total.head(1)


,city,cell_id,cell_pop,sky_d_bin,sky_d,walk_d,walk_t,bike_d,bike_t,pt_r_d,pt_r_t,pt_r_it,car_r_d,car_r_t,pt_m_d,pt_m_t,pt_m_it,car_m_d,car_m_t,geometry
0,Vienna,0,160.614578,9.5,9.995487,11.394,8204.0,11.9064,2418.8,11.8,2252.0,NaN,12.005,1629.0,11.8,2192.0,NaN,12.005,1334.0,"POLYGON ((16.33316 48.30187, 16.33268 48.29951..."
